In [ ]:
# default_exp cli


# CLI tools

As explained in [nbdev docs](https://nbdev.fast.ai/tutorial.html#Set-up-console-scripts) and in [the @call_parse docs](https://fastcore.fast.ai/script.html) we can have `nbdev` set up command line tools for us.


In [ ]:
# export
from pathlib import Path
from fastcore.script import call_parse
import json
import re
import io


A little script to correct chapter indices to base 1:


In [ ]:
# export
@call_parse
def rename(dir: str,  # dir in which to rename files
           suffix: str = ".ipynb"  # suffix of files to change
           ):
    """Rename notebooks to base 1 for chapters."""
    path = Path(dir)
    assert path.exists(), f"dir '{dir}' not found"
    assert path.is_dir(), f"'{dir}' is not a directory"
    print("Renaming notebooks now.")
    for ch in list(range(8, -1, -1)):
        print(f"Renaming chapter {ch}:")
        for x in path.glob(f"S{ch}*{suffix}"):
            new_name = path / x.name.replace(f"S{ch}", f"S{ch+1}")
            print(f"Renaming {x} to {new_name}")
            x.rename(new_name)


A function to read a notebook and fix its colab links:


In [ ]:
# export

REGEX = re.compile(
    '<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/(.+?)\.ipynb".*')


def _colab_link_name(nb):
    colab_link_source = nb['cells'][0]['source'][0]
    result = re.search(REGEX, colab_link_source)
    return result.group(1) if result is not None else None


def fix_colab_links(nb: dict, name: str):
    "Fix colab links in `nb`"

    # Fix colab link
    before = _colab_link_name(nb)
    if before is not None:
        colab_link_source = nb['cells'][0]['source'][0]
        nb['cells'][0]['source'][0] = colab_link_source.replace(before, name)

    # Fix colab metadata
    if 'colab' in nb['metadata']:
        nb['metadata']['colab']['name'] = name + '.ipynb'


In [ ]:
# Let's read a notebook in the test folder
name = 'nb_with_colab_link'
nb = json.loads(open(f'test/{name}.ipynb', 'r', encoding='utf-8').read())

fix_colab_links(nb, 'nb_with_colab_link')

assert _colab_link_name(nb) == name
assert nb['metadata']['colab']['name'] == name + '.ipynb'

nb_with_colab_link.ipynb


A script to do this for all the notebooks


In [ ]:
# export
@call_parse
def fix_nbs_colab_links(dir: str,  # dir in which to rename files
                        ):
    """Fix colab links in all notebooks in directory."""
    path = Path(dir)
    assert path.exists(), f"dir '{dir}' not found"
    assert path.is_dir(), f"'{dir}' is not a directory"
    print("Fixing notebooks now.")
    for x in path.glob("*.ipynb"):
        nb = json.loads(open(str(x), 'r', encoding='utf-8').read())
        fix_colab_links(nb, str(x.stem))
        all = json.dumps(nb, sort_keys=True, indent=1, ensure_ascii=False)
        with io.open(x, 'w', encoding='utf-8') as f:
            f.write(all)
            f.write("\n")
